# Definitions 

In [ ]:
# (https://www.xilinx.com/support/documentation/user_guides/ug576-ultrascale-gth-transceivers.pdf)
columns = ['DRP Address (Hex)','DRP Bits','R/W','Attribute Name','Attribute Bits','Attribute Encoding','DRP Encoding']

# p373 and following
filepath_channel = 'drp/u576_p373.csv'
# p352 and follwing
filepath_common = 'drp/u576_p352.csv'

In [ ]:
import pandas as pd
import re

single = re.compile(r'\[(\d*)\]')
double = re.compile(r'\[(\d*):(\d*)\]')

# Parse file

In [ ]:
df_channel = pd.read_csv(filepath_channel, sep=";", names=columns)
df_common = pd.read_csv(filepath_common, sep=";", names=columns)

# Channel

In [ ]:
df = df_channel
name = 'channel'
outputfile = f'../../modules/board_support_software/software/py/ru_gthe3_{name}_drp_mapping.py'

### Handle duplicated cells

In [ ]:
duplicates = ['RX_PRBS_ERR_CNT']

prev_nam = None
for index, row in df.iterrows():
    nam = row[columns[3]]
    if nam in duplicates:
        if nam != prev_nam:
            df.iloc[index][columns[3]] += '_LOW'
        else:
            df.iloc[index][columns[3]] += '_HIGH'
        print(df.iloc[index][columns[3]])
    prev_nam = nam

### Create File Header

In [ ]:
outstr = ""
outstr += f'\"\"\"Memory Mapping for the DRP port of the {name} auto-generated from\n'
outstr += f'https://www.xilinx.com/support/documentation/user_guides/ug576-ultrascale-gth-transceivers.pdf P373\n'
outstr += f'UG576 (v1.6) August 26, 2019\"\"\"\n'
outstr += '\n'
outstr += 'from aenum import Enum, NoAlias\n'
outstr += 'from enum import IntEnum\n'
outstr += '\n'
with open(outputfile,'w') as f:
    f.write(outstr)

### Create memory mapping

In [ ]:
outstr_na = ""
outstr_wa = ""

prev_nam, prev_add = None, None

outstr_na += "class Gthe3ChannelAddress(Enum):\n"
outstr_na += "    \"\"\"\n"
outstr_na += "    Class implementing the address as distinct elements.\n"
outstr_na += "    Iteration will run through all addresses.\n"
outstr_na += "    \"\"\"\n"
outstr_na += "    _settings_ = NoAlias\n"
outstr_na += '\n'

outstr_wa += "\n"   
outstr_wa += "\n"
outstr_wa += "class Gthe3ChannelAddressAliases(IntEnum):\n"
outstr_wa += "    \"\"\"\n"
outstr_wa += "    Class implementing the address as distinct elements.\n"
outstr_wa += "    Iteration will run through each address only once.\n"
outstr_wa += "    \"\"\"\n"

for index, row in df.iterrows():
    nam = row[columns[3]]
    add = int(row[columns[0]][:-1],16)
    if (nam,add) != (prev_nam, prev_add):
        msg = f"    {nam} = {add}\n"
        outstr_na += msg
        outstr_wa += msg
    prev_nam = nam
    prev_add = add

with open(outputfile,'a') as f:
    f.write(outstr_na)
    f.write(outstr_wa)

In [ ]:
outstr_l = ""
outstr_w = ""

prev_nam, prev_bits = None, None
outstr_l += "\n"   
outstr_l += "\n"
outstr_l += "class Gthe3ChannelLow(Enum):\n"
outstr_l += "    \"\"\"\n"
outstr_l += "    Class indicating the lowest significant bit of a field\n"
outstr_l += "    Iteration will run through all addresses.\n"
outstr_l += "    \"\"\"\n"
outstr_l += "    _settings_ = NoAlias\n"
outstr_l += '\n'

outstr_w += '\n'
outstr_w += "\n"
outstr_w += "class Gthe3ChannelWidth(Enum):\n"
outstr_w += "    \"\"\"\n"
outstr_w += "    Class indicating the width of a field\n"
outstr_w += "    Iteration will run through all addresses.\n"
outstr_w += "    \"\"\"\n"
outstr_w += "    _settings_ = NoAlias\n"
outstr_w += '\n'

for index, row in df.iterrows():
    nam = row[columns[3]]
    bits = row[columns[1]]
    if (nam,bits) != (prev_nam, prev_bits):
        m = single.match(bits)
        if m:
            low = int(m.groups()[0])
            assert low in range(16)
            width = 1
            outstr_l += f"    {nam} = {low}\n"
            outstr_w += f"    {nam} = {width}\n" # width is 1 in this case
        else:
            m = double.match(bits)
            if m:
                low = int(m.groups()[1])
                high = int(m.groups()[0])
                assert high > low
                assert low in range(16)
                assert high in range(16)
                width = high - low + 1
                outstr_l += f"    {nam} = {low}\n"
                outstr_w += f"    {nam} = {width}\n"
            else:
                print('NOT FOUND!')
    prev_nam = nam
    prev_bits = bits


with open(outputfile,'a') as f:
    f.write(outstr_l)
    f.write(outstr_w)

# Common

In [ ]:
df = df_common
name = 'common'
outputfile = f'../../modules/board_support_software/software/py/ru_gthe3_{name}_drp_mapping.py'

### Handle duplicated cells

In [ ]:
duplicates = []

prev_nam = None
for index, row in df.iterrows():
    nam = row[columns[3]]
    if nam in duplicates:
        if nam != prev_nam:
            df.iloc[index][columns[3]] += '_LOW'
        else:
            df.iloc[index][columns[3]] += '_HIGH'
        print(df.iloc[index][columns[3]])
    prev_nam = nam

### Create File Header

In [ ]:
outstr = ""
outstr += f'\"\"\"Memory Mapping for the DRP port of the {name} auto-generated from\n'
outstr += f'https://www.xilinx.com/support/documentation/user_guides/ug576-ultrascale-gth-transceivers.pdf P352\n'
outstr += f'UG576 (v1.6) August 26, 2019\"\"\"\n'
outstr += '\n'
outstr += 'from aenum import Enum, NoAlias\n'
outstr += 'from enum import IntEnum\n'
outstr += '\n'
with open(outputfile,'w') as f:
    f.write(outstr)

### Create memory mapping

In [ ]:
outstr_na = ""
outstr_wa = ""

prev_nam, prev_add = None, None

outstr_na += "class Gthe3CommonAddress(Enum):\n"
outstr_na += "    \"\"\"\n"
outstr_na += "    Class implementing the address as distinct elements.\n"
outstr_na += "    Iteration will run through all addresses.\n"
outstr_na += "    \"\"\"\n"
outstr_na += "    _settings_ = NoAlias\n"
outstr_na += '\n'

outstr_wa += "\n"   
outstr_wa += "\n"
outstr_wa += "class Gthe3CommonAddressAliases(IntEnum):\n"
outstr_wa += "    \"\"\"\n"
outstr_wa += "    Class implementing the address as distinct elements.\n"
outstr_wa += "    Iteration will run through each address only once.\n"
outstr_wa += "    \"\"\"\n"

for index, row in df.iterrows():
    nam = row[columns[3]]
    add = int(row[columns[0]][:-1],16)
    if (nam,add) != (prev_nam, prev_add):
        msg = f"    {nam} = {add}\n"
        outstr_na += msg
        outstr_wa += msg
    prev_nam = nam
    prev_add = add

with open(outputfile,'a') as f:
    f.write(outstr_na)
    f.write(outstr_wa)

In [ ]:
outstr_l = ""
outstr_w = ""

prev_nam, prev_bits = None, None
outstr_l += "\n"   
outstr_l += "\n"
outstr_l += "class Gthe3CommonLow(Enum):\n"
outstr_l += "    \"\"\"\n"
outstr_l += "    Class indicating the lowest significant bit of a field\n"
outstr_l += "    Iteration will run through all addresses.\n"
outstr_l += "    \"\"\"\n"
outstr_l += "    _settings_ = NoAlias\n"
outstr_l += '\n'

outstr_w += '\n'
outstr_w += "\n"
outstr_w += "class Gthe3CommonWidth(Enum):\n"
outstr_w += "    \"\"\"\n"
outstr_w += "    Class indicating the width of a field\n"
outstr_w += "    Iteration will run through all addresses.\n"
outstr_w += "    \"\"\"\n"
outstr_w += "    _settings_ = NoAlias\n"
outstr_w += '\n'

for index, row in df.iterrows():
    nam = row[columns[3]]
    bits = row[columns[1]]
    if (nam,bits) != (prev_nam, prev_bits):
        m = single.match(bits)
        if m:
            low = int(m.groups()[0])
            assert low in range(16)
            width = 1
            outstr_l += f"    {nam} = {low}\n"
            outstr_w += f"    {nam} = {width}\n" # width is 1 in this case
        else:
            m = double.match(bits)
            if m:
                low = int(m.groups()[1])
                high = int(m.groups()[0])
                assert high > low
                assert low in range(16)
                assert high in range(16)
                width = high - low + 1
                outstr_l += f"    {nam} = {low}\n"
                outstr_w += f"    {nam} = {width}\n"
            else:
                print('NOT FOUND!')
    prev_nam = nam
    prev_bits = bits


with open(outputfile,'a') as f:
    f.write(outstr_l)
    f.write(outstr_w)